## A python stream generating application

In [1]:
import time
import csv
import socket
import uuid
from multiprocessing import Process

with open('./ClimateData-Part2.csv','r') as c2, open ('./FireData-Part2.csv','r') as f2:
    climate_reader = csv.reader(c2)
    fire_reader  = csv.reader(f2)
    climate = []
    fire  = []
    for row in climate_reader:
        climate.append(','.join(e.strip() for e in row) + '\n ')
    for line in fire_reader:
        line.append(uuid.uuid1().hex)
        fire.append(','.join(e.strip() for e in line) + '\n')
climate =  climate[1:]
fire = fire[1:]  
def generator(dataset):
    for e in dataset:
        yield e
climate_genrator = generator(climate)
fire_genrator = generator(fire)
def generate_stream(dataset,port):
    """
    The method that generate a stream sending to socket of the computer from a given dataset
    -------
    Parameters:
    dataset ---the generator object of the dataset
    port --- the usable port that the data stream can be sent through
    """ 
    host = 'localhost'

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    s.bind((host, port))
    s.listen(1)
    i = 0
    while True:
        try:
            conn, addr = s.accept()
            print(i)
            i += 1
            while True:
                try:
                    conn.send(next(dataset).encode())
                    time.sleep(1)
                except StopIteration:
                    s.close()
                    return
        except socket.error: 
            pass
    conn.close()
    s.close()
    return
                

if __name__ == '__main__':
    #here we must open the two sockets using mp so that it will be openn at the same time
    #otherwise the other port cannot be connected to pyspark DStream
    p1 = Process(target=generate_stream, args=(climate_genrator,9999))
    p2 = Process(target=generate_stream, args=(fire_genrator,8080))
    p1.start()
    p2.start()

0
0


### Here we should implement mainly two functions:
1. A method that can take 5 records and 1 record from both dataset per second
2. Sent the retrieved data as text to 

In [3]:
from multiprocessing import Process

In [ ]:
Process(cil)

In [ ]:
with open('./ClimateData-Part2.csv','r') as c2, open ('./FireData-Part2.csv','r') as f2:
    climate_reader = csv.reader(c2)
    fire_reader  = csv.reader(f2)
    climate = []
    fire  = []
    for row in climate_reader:
        climate.append(','.join(e.strip() for e in row) + '\n ')
    for line in fire_reader:
        line.append(uuid.uuid1().hex)
        fire.append(','.join(e.strip() for e in line) + '\n')



In [ ]:
climate =  climate[1:]
fire = fire[1:]

In [ ]:
climate

In [ ]:
def generator(dataset):
    for e in dataset:
        yield e
c = generator(climate)
f = generator(fire)

In [ ]:
import socket
import time

host = 'localhost'
port = 9999

i = 0

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

s.bind((host, port))
s.listen(1)

try:
    while True:
        conn, addr = s.accept()
        try:
            for j in range(10):
                conn.send(next(c).encode())
                i += 1
                time.sleep(1)
            conn.close()
        except socket.error: pass
finally:
    s.close()



In [ ]:
len(climate)

In [ ]:
len(fire)//5 +1

#### another question is that, everything send through socket is byte, is it possible that we reverse it back to list and retain its original information?
- String willbe the easiest to encode and decode
#### If we save each record just in a string sperated by ",", then the encode and decode will be much easier

In [ ]:
# list to json str
t = json.dumps(next(a))

In [ ]:
# json str to byte and back to json str
t = t.encode()
t.decode()

In [ ]:
# load json str back to list
json.loads(t)[0]

In [ ]:
def generate_streams():
    pass

In [ ]:
import socket

In [ ]:
import socket

def netcat(host, port, content):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, int(port)))
    s.sendall(content.encode())
    s.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(4096)
        if not data:
            break
        print(repr(data))
    s.close()

In [ ]:
netcat('localhost',9999,'aa aa aa aaaa')

In [ ]:
def a():
    for e in range(3):
        yield e

In [ ]:
g =a()

In [ ]:
while True:
    try:
        print(next(g))
    except StopIteration:
        print('done')
        break
    finally:
        print('yes')


### we need to create two socket connection for the two datasets to generate two streams